In [1]:
import dash_cytoscape as cyto
import graph_tool.all as gt
import graph_tool.util as gu
import os

from dash import Dash, dcc, html, Input, Output, State
from graph_tool import GraphView
from jupyter_dash import JupyterDash
from tqdm import tqdm

In [2]:
graph_list=os.listdir('../data/graph-tool/')[:-1]
g = gt.load_graph("../data/graph-tool/specificCelltypeGraph_B_S00DFM11.gt")
gen_list = [a for a in list(g.vp.name) if a.startswith("E")]
# rem_list = [a for a in list(g.vp.name) if a.startswith("R")]
cyto_list = []

In [3]:
def get_network(node):
    v = gu.find_vertex(g, g.vp.name, node)[0]
    nodes = []
    edges= []
    vfilt = g.new_vertex_property('bool')
    vfilt[v] = True
    nodes.append({'data': {'id': g.vp.name[v], 'label': g.vp.name[v]}, 'classes':'green'})
    for e, w in zip(v.in_edges(), v.in_neighbors()):
        vfilt[w] = True
        nodes.append({'data': {'id': g.vp.name[w], 'label': g.vp.name[w]}, 'classes': 'red triangle'})
        edges.append({'data': {'source': g.vp.name[e.source()], 'target': g.vp.name[e.target()], 'label': g.ep.standDnase1Log2[e]}})

    sub = GraphView(g, vfilt)
    cyto_list = nodes+edges
    return cyto_list

In [4]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H1("REMnet"),
    dcc.Dropdown(
        id='dropdown-graph',
        value=graph_list[0],
        clearable=False,
        options=[
            {'label': name, 'value': name}
            for name in graph_list
        ]
    ),
    dcc.Dropdown(
        id='dropdown-gen',
        value=gen_list[0],
        clearable=True,
        options=[
            {'label': name, 'value': name}
            for name in gen_list
        ]
    ),
#     dcc.Dropdown(
#         id='dropdown-rem',
#         value=rem_list[0],
#         clearable=True,
#         options=[
#             {'label': name, 'value': name}
#             for name in rem_list
#         ]
#     ),
    cyto.Cytoscape(
        id='network-graph',
        layout={'name': 'circle'},
        style={'width': '100%', 'height': '700px'},
        elements=cyto_list,
        stylesheet=[
#             # Group selectors
            {
                'selector': 'node',
                'style': {
                    'content': 'data(label)'
                }
            },
            {
                'selector': 'edge',
                'style': {
                    'content': 'data(label)'
                }
            },

            # Class selectors
            {
                'selector': '.red',
                'style': {
                    'background-color': 'red',
                    'line-color': 'red'
                }
            },
            {
                'selector': '.green',
                'style': {
                    'background-color': 'green',
                    'line-color': 'green'
                }
            },
            {
                'selector': '.triangle',
                'style': {
                    'shape': 'triangle'
                }
            }
        ]
    ),
    html.Br(),
    html.Div(id='body-div')
])

@app.callback(Output('network-graph', 'elements'),
              Input('dropdown-gen', 'value'),
              Input('dropdown-graph', 'value1'),
              State('network-graph', 'elements'))
def update_elements(value, value1, elements):
    return get_network(value)

@app.callback(Output('body-div', 'children'),
              Input('dropdown-graph', 'input_value'))
def update_file(input_value):
    
    return input_value

#     g = gt.load_graph("../data/graph-tool/"+value)
#     gen_list = [a for a in list(g.vp.name) if a.startswith("E")]
#     cyto_list = []
    

# @app.callback(Output('network-graph', 'elements'),
#               Input('dropdown-rem', 'value'),
#               State('network-graph', 'elements'))
# def update_elements(value, elements):
#     return get_network(value)

app.run_server(host='0.0.0.0', port=8050, debug=False)

 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8050
 * Running on http://10.162.163.34:8050
Press CTRL+C to quit
127.0.0.1 - - [17/Jan/2023 14:04:12] "GET /_alive_cd9c1899-6b55-4c13-9157-626dac69942b HTTP/1.1" 200 -


Dash app running on http://0.0.0.0:8050/


10.157.36.211 - - [17/Jan/2023 14:04:20] "GET / HTTP/1.1" 200 -
10.157.36.211 - - [17/Jan/2023 14:04:20] "GET /_favicon.ico?v=2.7.0 HTTP/1.1" 200 -
10.157.36.211 - - [17/Jan/2023 14:04:20] "GET /_dash-dependencies HTTP/1.1" 200 -
10.157.36.211 - - [17/Jan/2023 14:04:20] "GET /_dash-layout HTTP/1.1" 200 -
10.157.36.211 - - [17/Jan/2023 14:04:21] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
10.157.36.211 - - [17/Jan/2023 14:04:21] "POST /_dash-update-component HTTP/1.1" 200 -
10.157.36.211 - - [17/Jan/2023 14:04:21] "POST /_dash-update-component HTTP/1.1" 200 -
10.157.36.211 - - [17/Jan/2023 14:05:20] "POST /_dash-update-component HTTP/1.1" 200 -
10.157.36.211 - - [17/Jan/2023 14:06:20] "POST /_dash-update-component HTTP/1.1" 500 -
10.157.36.211 - - [17/Jan/2023 14:06:21] "POST /_dash-update-component HTTP/1.1" 200 -
10.157.36.211 - - [17/Jan/2023 14:06:31] "POST /_dash-update-component HTTP/1.1" 200 -
10.157.36.211 - - [17/Jan/2023 14:16:49] "POST /_dash-updat

In [ ]:
import matplotlib.pyplot as plt

weights = [g.ep.standDnase1Log2[e] for e in g.edges()]
plt.hist(weights)
plt.xlabel('weight')
# plt.show()

In [ ]:
v = gu.find_vertex(g, g.vp.name, "ENSG00000223972")[0]
nodes = []
edges= []
vfilt = g.new_vertex_property('bool')
vfilt[v] = True
nodes.append({'data': {'id': g.vp.name[v], 'label': g.vp.name[v]}, 'classes':'green'})
for e, w in zip(v.in_edges(), v.in_neighbors()):
    vfilt[w] = True
    nodes.append({'data': {'id': g.vp.name[w], 'label': g.vp.name[w]}, 'classes': 'red triangle'})
    edges.append({'data': {'source': g.vp.name[e.source()], 'target': g.vp.name[e.target()], 'label': g.ep.standDnase1Log2[e]}})

sub = GraphView(g, vfilt)
cyto_list = nodes+edges